In [12]:
# load a coffea file
from coffea.util import load
import os
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib
import re
matplotlib.rcParams["figure.dpi"] = 300


# Load file

In [13]:
# input = "/work/rcereghetti/out_vbfhh4b/output_vbf_matching3/output_all.coffea"
# o = load(f"{input}")
input = "/work/rcereghetti/out_vbfhh4b/out_test/output_all.coffea"
o = load(f"{input}")

# Function to extract the cut string

In [14]:
def remove_if_found(word, substring):
    if substring in word:
        return word.replace(substring, '')
    return word  # Restituisce la parola invariata se la sottostringa non è trovata

# Create a dictionary with number of events for every step and region

In [15]:
cuts_dict = {}

pattern = r'^[^_]+_[^_]+_([^_]+)_region'

for k in o["cutflow"].keys(): 
    print(k)   
    if "VBFtight" in k and "VBFtight_region" not in k:
        a = remove_if_found(k, "4b_")
        b = remove_if_found(a, "_region")
        c = remove_if_found(b, "VBFtight_")
        string = c
    else: string = k
    for kk in o["cutflow"][k].keys():
        print(o['cutflow'][k][kk])
        if isinstance(o['cutflow'][k][kk], int):
            cuts_dict[string] = o['cutflow'][k][kk]
        else:
            cuts_dict[string] = o['cutflow'][k][kk]['VBF_HHto4B']


initial
204633
skim
44944
presel
31646
4b_region
{'VBF_HHto4B': 8576}
4b_VBFtight_njet_vbf_region
{'VBF_HHto4B': 5083}


# b

In [16]:
cuts_list = list(cuts_dict)

for i in range(4, len(cuts_list)):
    print(f"Efficiency for {cuts_list[i]} with respect to 4b_region: {cuts_dict[cuts_list[i]] / cuts_dict['4b_region']}")

Efficiency for njet_vbf with respect to 4b_region: 0.5927005597014925
